In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')

# Basic inspection
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print("\nTrain columns:")
print(train.dtypes)

# Handle TransactionStartTime
train['TransactionStartTime'] = pd.to_datetime(train['TransactionStartTime'])
test['TransactionStartTime'] = pd.to_datetime(test['TransactionStartTime'])

# Key visualizations
plt.figure(figsize=(12,6))
sns.countplot(x=train['ProductCategory'])
plt.title('Product Category Distribution')
plt.xticks(rotation=45)
plt.savefig('../reports/product_category_dist.png')

# Top 3 insights
insights = [
    "1. High concentration in Electronics category (65% of transactions)",
    "2. 12% missing values in PricingStrategy column",
    "3. Significant outliers in Amount column (transactions up to 1.5M)"
]